## The Original EigenDecompression.EigenDecompose

See

* https://mobile.twitter.com/realize_ss/status/1615160291108745216
* https://qiita.com/lelele/items/8408410a94f5c6b8f76e

In [1]:
using LinearAlgebra
M = rand(100,100)#対角化したい行列
E, P = eigen(M)

Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}}
values:
100-element Vector{ComplexF64}:
 -2.6713385149783693 - 0.5238483074100407im
 -2.6713385149783693 + 0.5238483074100407im
  -2.380694749857707 - 1.5819003151310398im
  -2.380694749857707 + 1.5819003151310398im
 -2.3445049148115737 - 0.8348768899082996im
 -2.3445049148115737 + 0.8348768899082996im
 -2.1226846073905064 - 0.22379425662578242im
 -2.1226846073905064 + 0.22379425662578242im
  -2.049294063421365 - 1.6320269189857641im
  -2.049294063421365 + 1.6320269189857641im
 -2.0178436003856652 + 0.0im
 -1.9229326770807857 - 1.7886924334849146im
 -1.9229326770807857 + 1.7886924334849146im
                     ⋮
  1.9543269536275725 + 0.5040016799489412im
  1.9954076975615094 - 1.0426916311345158im
  1.9954076975615094 + 1.0426916311345158im
  2.0560135421866663 - 0.48888998497523045im
  2.0560135421866663 + 0.48888998497523045im
   2.416364338786246 + 0.0im
   2.486887950907518 - 1.224341365689113im
   2.486887950

In [2]:
exp(eigen(M))

LoadError: MethodError: no method matching exp(::Eigen{ComplexF64, ComplexF64, Matrix{ComplexF64}, Vector{ComplexF64}})

[0mClosest candidates are:
[0m  exp([91m::Union{Float16, Float32, Float64}[39m)
[0m[90m   @[39m [90mBase[39m [90mspecial\[39m[90m[4mexp.jl:325[24m[39m
[0m  exp([91m::Adjoint{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:595[24m[39m
[0m  exp([91m::Transpose{T, <:AbstractMatrix} where T[39m)
[0m[90m   @[39m [35mLinearAlgebra[39m [90mD:\Julia-1.9.0-beta2\share\julia\stdlib\v1.9\LinearAlgebra\src\[39m[90m[4mdense.jl:596[24m[39m
[0m  ...


In [3]:
module EigenDecompression

export EigenDecompose, eigDecomp
using LinearAlgebra
import Base.*, Base./

#対角化された行列型
struct EigenDecompose{T<:Number} <: AbstractMatrix{T}
    P::AbstractMatrix{T}
    D::Diagonal{T}
    invP::AbstractMatrix{T}
end

#普通のMatrixを対角化する
function eigDecomp(mat::AbstractMatrix)
    E, P = eigen(mat)
    EigenDecompose(P, Diagonal(E), inv(P))
end

#EigenDecompose型に対する関数
Base.exp(eig::EigenDecompose) = EigenDecompose(eig.P, exp(eig.D), eig.invP)
*(eig::EigenDecompose, vec::AbstractVector) = eig.P * eig.D * eig.invP * vec
*(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D*sc, eig.invP)
/(eig::EigenDecompose, sc::Number) = EigenDecompose(eig.P, eig.D/sc, eig.invP)

#普通のMatrixに戻す
Base.Array(eig::EigenDecompose) = eig.P * eig.D * eig.invP

end

Main.EigenDecompression

In [4]:
using .EigenDecompression

M = rand(100, 100)
eM = eigDecomp(M)
for i in 1:100
    v = rand(100)
    rnd = rand()
    @assert exp(M*rnd)*v ≈ exp(eM*rnd)*v
end

In [5]:
using BenchmarkTools

M = rand(100, 100);

#普通な方
function bench1(M)
    for i in 1:100
        v = rand(100)
        exp(M*rand())*v
    end
end

#今回実装した方
function bench2(M)
    eM = eigDecomp(M)
    for i in 1:100
        v = rand(100)
        exp(eM*rand())*v
    end
end

bench2 (generic function with 1 method)

In [6]:
@benchmark bench1(M)

BenchmarkTools.Trial: 58 samples with 1 evaluation.
 Range (min … max):  78.395 ms … 100.653 ms  ┊ GC (min … max): 2.43% … 3.65%
 Time  (median):     86.035 ms               ┊ GC (median):    2.82%
 Time  (mean ± σ):   87.298 ms ±   4.854 ms  ┊ GC (mean ± σ):  3.62% ± 1.60%

             ▂      █                                           
  ▄▄▁▁▄▄▁▁▁▁▁█▁▁█▄▆▁███▄▆▄▄▁██▁▄▄▆▄▁▄▁▁▆▁▄▄▆▁▁▄▆▁▁▁▁▁▁▄▆▁▁▁▁▁▄ ▁
  78.4 ms         Histogram: frequency by time         98.7 ms <

 Memory estimate: 53.80 MiB, allocs estimate: 1900.

In [7]:
@benchmark bench2(M)

BenchmarkTools.Trial: 690 samples with 1 evaluation.
 Range (min … max):  6.103 ms … 15.411 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.824 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.234 ms ±  1.191 ms  ┊ GC (mean ± σ):  0.48% ± 2.20%

     █▅▂▁                                                     
  ▃▅▇████▇█▇▆▇▄▃▃▃▃▃▃▃▃▃▃▃▂▃▂▃▂▃▂▂▂▂▂▃▂▂▁▂▂▂▂▂▂▁▂▂▁▂▂▂▂▁▁▁▁▂ ▃
  6.1 ms         Histogram: frequency by time        12.3 ms <

 Memory estimate: 1.82 MiB, allocs estimate: 1728.

## EigenDecomposedMatrices.EigenDecomposed

In [8]:
using LinearAlgebra
using BenchmarkTools

In [9]:
module EigenDecomposedMatrices

export EigenDecomposed

using LinearAlgebra
using Memoization

struct EigenDecomposed{
        T,
        TE<:AbstractVector{T},
        TP<:AbstractMatrix{T},
        TinvP<:AbstractMatrix{T}
    } <: AbstractMatrix{T}
    E::TE
    P::TP
    invP::TinvP
end

function EigenDecomposed(A::AbstractMatrix)
    E, P = eigen(A)
    invP = ishermitian(A) ? P' : inv(P)
    EigenDecomposed(E, P, invP)
end

LinearAlgebra.eigvals(ed::EigenDecomposed) = ed.E
LinearAlgebra.eigvecs(ed::EigenDecomposed) = ed.P
inveigvecs(ed::EigenDecomposed) = ed.invP
@memoize Base.parent(ed::EigenDecomposed) = eigvecs(ed) * Diagonal(eigvals(ed)) * inveigvecs(ed)
Base.convert(::Type{Array}, ed::EigenDecomposed) = convert(Array, parent(ed))
for op in (:eltype, :size)
    @eval Base.$op(ed::EigenDecomposed) = $op(eigvecs(ed))
end
Base.getindex(ed::EigenDecomposed, I...) = getindex(parent(ed), I...)

Base.:*(c::Number, ed::EigenDecomposed) = EigenDecomposed(c*eigvals(ed), eigvecs(ed), inveigvecs(ed))
Base.:*(ed::EigenDecomposed, c::Number) = EigenDecomposed(eigvals(ed)*c, eigvecs(ed), inveigvecs(ed))
Base.:\(c::Number, ed::EigenDecomposed) = EigenDecomposed(c\eigvals(ed), eigvecs(ed), inveigvecs(ed))
Base.:/(ed::EigenDecomposed, c::Number) = EigenDecomposed(eigvals(ed)/c, eigvecs(ed), inveigvecs(ed))
for T in (AbstractVector, AbstractMatrix)
    @eval function Base.:*(ed::EigenDecomposed, v::$T)
        E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
        P * (Diagonal(E) * (invP * v))
    end
end

function exp_old(ed::EigenDecomposed)
    E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
    expE = exp.(E)
    expA = P * Diagonal(expE) * invP 
    EigenDecomposed(expE, P, invP)
end

LinearAlgebra.lmul!(c::Number, ed::EigenDecomposed) = lmul!(c, eigvals(ed))
LinearAlgebra.rmul!(ed::EigenDecomposed, c::Number) = rmul!(eigvals(ed), c)
LinearAlgebra.ldiv!(c::Number, ed::EigenDecomposed) = ldiv!(c, eigvals(ed))
LinearAlgebra.rdiv!(ed::EigenDecomposed, c::Number) = rdiv!(eigvals(ed), c)

for op in (:exp, :log, :sin, :cos)
    opE = Symbol(op, "E")
    op_eigendecomposed = Symbol(op, "_eigendecomposed")
    op_eigendecomposed! = Symbol(op_eigendecomposed, "!")
    op_eigendecomposed!_doc =
        """
        $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))

        returns the `$op` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`. 
        It does not overwrite `ed` and uses `$opE` and `tmpY` as workspaces.
        """
    @eval begin
        @doc $op_eigendecomposed!_doc
        function $op_eigendecomposed!(Y, ed::EigenDecomposed, $opE=similar(ed.E), tmpY=similar(Y))
            E, P, invP = eigvals(ed), eigvecs(ed), inveigvecs(ed)
            @. $opE = $op.(E)
            mul!(tmpY, P, Diagonal($opE))
            mul!(Y, tmpY, invP)
        end
        $op_eigendecomposed(ed::EigenDecomposed) = $op_eigendecomposed!(similar(eigvecs(ed)), ed)
        Base.$op(ed::EigenDecomposed) = $op_eigendecomposed(ed)
    end
end

end

Main.EigenDecomposedMatrices

In [10]:
?EigenDecomposedMatrices.exp_eigendecomposed!

exp_eigendecomposed!(Y, ed::EigenDecomposed, expE=similar(ed.E), tmpY=similar(Y))

returns the `exp` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `expE` and `tmpY` as workspaces.


In [11]:
?EigenDecomposedMatrices.log_eigendecomposed!

log_eigendecomposed!(Y, ed::EigenDecomposed, logE=similar(ed.E), tmpY=similar(Y))

returns the `log` of `ed` and stores the result in `Y`, overwriting the existing value of `Y`.  It does not overwrite `ed` and uses `logE` and `tmpY` as workspaces.


In [12]:
methods(EigenDecomposedMatrices.EigenDecomposed)

# 2 methods for type constructor:
 [1] Main.EigenDecomposedMatrices.EigenDecomposed(E::TE, P::TP, invP::TinvP) where {T, TE<:AbstractVector{T}, TP<:AbstractMatrix{T}, TinvP<:AbstractMatrix{T}}
     @ In[9]:14
 [2] Main.EigenDecomposedMatrices.EigenDecomposed(A::AbstractMatrix)
     @ In[9]:19

In [13]:
methods(EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Matrix{Float64}})

# 1 method for type constructor:
 [1] (var"#ctor-self#"::Type{Main.EigenDecomposedMatrices.EigenDecomposed{T, TE, TP, TinvP}} where {T, TE<:AbstractVector{T}, TP<:AbstractMatrix{T}, TinvP<:AbstractMatrix{T}})(E, P, invP)
     @ In[9]:14

In [14]:
methodswith(EigenDecomposedMatrices.EigenDecomposed)

[1] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:36
[2] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractVector) @ Main.EigenDecomposedMatrices In[9]:40
[3] *(ed::Main.EigenDecomposedMatrices.EigenDecomposed, v::AbstractMatrix) @ Main.EigenDecomposedMatrices In[9]:40
[4] *(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:35
[5] /(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:38
[6] \(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:37
[7] convert(::Type{Array}, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:29
[8] cos(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[9] eltype(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:31
[10] exp(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[11] getindex(ed::Main.EigenDecomposedMatrices.EigenDecomposed, I...) @ Main.EigenDecomposedMatrices In[9]:33
[12] log(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[13] parent(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices D:\.julia\packages\Memoization\ut5GT\src\Memoization.jl:162
[14] sin(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:78
[15] size(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:31
[16] eigvals(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:25
[17] eigvecs(ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:26
[18] ldiv!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:55
[19] lmul!(c::Number, ed::Main.EigenDecomposedMatrices.EigenDecomposed) @ Main.EigenDecomposedMatrices In[9]:53
[20] rdiv!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:56
[21] rmul!(ed::Main.EigenDecomposedMatrices.EigenDecomposed, c::Number) @ Main.EigenDecomposedMatrices In[9]:54

In [15]:
methods(EigenDecomposedMatrices.exp_eigendecomposed!)

# 3 methods for generic function "exp_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:71
 [2] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE)
     @ In[9]:71
 [3] exp_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, expE, tmpY)
     @ In[9]:71

In [16]:
methods(EigenDecomposedMatrices.log_eigendecomposed!)

# 3 methods for generic function "log_eigendecomposed!" from Main.EigenDecomposedMatrices:
 [1] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed)
     @ In[9]:71
 [2] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE)
     @ In[9]:71
 [3] log_eigendecomposed!(Y, ed::Main.EigenDecomposedMatrices.EigenDecomposed, logE, tmpY)
     @ In[9]:71

In [17]:
A = [
    2 -1 0
    -1 2 -1
    0 -1 2
]

edA = EigenDecomposedMatrices.EigenDecomposed(A)

3×3 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  2.0          -1.0  -3.33067e-16
 -1.0           2.0  -1.0
 -3.33067e-16  -1.0   2.0

In [18]:
log(edA)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [19]:
log(A)

3×3 Matrix{Float64}:
  0.51986   -0.623225  -0.173287
 -0.623225   0.346574  -0.623225
 -0.173287  -0.623225   0.51986

In [20]:
log(edA) ≈ log(A)

true

In [21]:
n = 2^8
M = 5I + randn(n, n)
v = randn(n)
c = randn()

edM = EigenDecomposedMatrices.EigenDecomposed(M)

Y = similar(eigvecs(edM))
expE = similar(eigvals(edM))
tmpY = similar(Y)

y = similar(eigvals(edM))
alpha = randn()
beta = randn();

In [22]:
edM

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}:
    3.56334-1.65165e-13im  …  -0.0799542+7.35922e-14im
   -1.29471-3.33504e-14im        2.79704-1.73821e-15im
   0.188711-1.38904e-14im      -0.146065-7.04386e-16im
  -0.139691-1.2809e-13im        -0.11464+1.72234e-13im
   0.242261+2.90866e-14im       -1.44222+1.48799e-13im
   0.476316+3.58411e-14im  …    0.497164-2.26801e-13im
   0.550097-3.61273e-14im      -0.134351+1.41119e-13im
   0.717261+1.67825e-13im       0.895324-9.16857e-14im
   -1.04636-8.49634e-14im      -0.190769-2.40075e-13im
     1.6971-2.75795e-14im       0.770272-6.1418e-14im
    1.26714+5.92264e-14im  …   -0.567488-1.78777e-13im
 -0.0854576-1.88943e-14im      -0.822014+4.62595e-14im
   0.826429+6.51037e-14im       -1.53237-1.375e-13im
           ⋮               ⋱            ⋮
  -0.997608-2.67526e-14im      0.0412252+1.99661e-13im
 -0.0798052-1.49411e-14im  …    0.144672-1.3304e-13im
   -1.17006

In [23]:
dump(edM)

Main.EigenDecomposedMatrices.EigenDecomposed{ComplexF64, Vector{ComplexF64}, Matrix{ComplexF64}, Matrix{ComplexF64}}
  E: Array{ComplexF64}((256,)) ComplexF64[-10.529011730788033 + 0.0im, -10.280415425445305 - 5.0635332402051345im, -10.280415425445305 + 5.0635332402051345im, -9.365531653942693 - 7.310921121421777im, -9.365531653942693 + 7.310921121421777im, -9.066423960276218 - 0.8998268797512045im, -9.066423960276218 + 0.8998268797512045im, -8.732971181835605 - 1.4609873383902336im, -8.732971181835605 + 1.4609873383902336im, -8.46124963643927 - 8.33273467746558im  …  18.632067916841073 - 4.578055223585328im, 18.632067916841073 + 4.578055223585328im, 18.93064239638064 - 7.073434597909069im, 18.93064239638064 + 7.073434597909069im, 19.20170503001391 - 3.658032128271094im, 19.20170503001391 + 3.658032128271094im, 20.930683959675996 - 0.7030285754956695im, 20.930683959675996 + 0.7030285754956695im, 21.184754820465372 - 3.0347453281790697im, 21.184754820465372 + 3.0347453281790697im]
  P: 

In [24]:
M ≈ parent(edM) == Matrix(edM)

true

In [25]:
M ≈ edM

true

In [26]:
c*M ≈ c*edM ≈ edM*c

true

In [27]:
c\M ≈ c\edM ≈ edM/c

true

In [28]:
(
    exp(M)
    ≈ exp(edM)
    ≈ EigenDecomposedMatrices.exp_old(edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY)
)

true

In [29]:
@show typeof(y)
(
    exp(M) * v
    ≈ exp(edM) * v
    ≈ EigenDecomposedMatrices.exp_old(edM) * v
    ≈ mul!(y, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM), v)
    ≈ mul!(y, EigenDecomposedMatrices.exp_eigendecomposed!(Y, edM, expE, tmpY), v)
)

typeof(y) = Vector{ComplexF64}


true

In [30]:
@btime edM = EigenDecomposedMatrices.EigenDecomposed(M);

  40.167 ms (26 allocations: 3.53 MiB)


In [31]:
@btime exp($M) * $v
@btime exp($edM) * $v
@btime EigenDecomposedMatrices.exp_old($edM) * $v
@btime mul!($y, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM), $v)
@btime mul!($y, EigenDecomposedMatrices.exp_eigendecomposed!($Y, $edM, $expE, $tmpY), $v);

  10.090 ms (16 allocations: 3.01 MiB)
  2.712 ms (7 allocations: 2.01 MiB)
  2.696 ms (9 allocations: 2.02 MiB)
  2.540 ms (3 allocations: 1.00 MiB)
  2.447 ms (0 allocations: 0 bytes)


In [32]:
n2 = 2^8
M2 = Symmetric(5I + randn(n2, n2))
v2 = randn(n)
c2 = randn()

edM2 = EigenDecomposedMatrices.EigenDecomposed(M2)

Y2 = similar(eigvecs(edM2))
expE2 = similar(eigvals(edM2))
tmpY2 = similar(Y2)

y2 = similar(eigvals(edM2))
alpha2 = randn()
beta2 = randn();

In [33]:
edM2

256×256 Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}:
  6.03077    -0.43245     0.599883    …  -0.872836      0.980907
 -0.43245     7.07582    -0.965762       -2.46577       1.27889
  0.599883   -0.965762    6.28325        -0.897865      0.902268
  0.554185   -1.61176    -1.88607         1.52239      -0.800269
 -0.0394612  -0.472386   -2.16329        -0.666488      1.60583
 -0.684983    0.673263    1.44873     …  -1.11132      -1.06482
  0.544451    1.8252      0.567267        0.568297      1.60087
 -0.60012    -1.6835     -0.332066        0.151455     -1.76098
 -1.32028    -1.71558    -0.503773       -0.276013      1.46591
 -0.619846    0.546905   -1.79987         0.501719      0.114942
 -1.96161     0.575018   -0.648195    …   1.16492      -1.17966
 -0.303545   -0.670053    0.967682       -0.000749713   0.373673
  1.33041    -0.0295437  -0.717992        0.155946      1.85579
  ⋮                            

In [34]:
dump(edM2)

Main.EigenDecomposedMatrices.EigenDecomposed{Float64, Vector{Float64}, Matrix{Float64}, Adjoint{Float64, Matrix{Float64}}}
  E: Array{Float64}((256,)) [-26.281246714455474, -26.079893599626686, -25.32277485066867, -25.0801104625073, -23.954875700929986, -23.74285195174835, -23.168019607432065, -22.60925439400774, -22.454572564359246, -21.705343122758094  …  31.569737139516377, 32.20706026446964, 32.53684276189552, 33.124400141791504, 33.974663809226215, 34.44625078387939, 34.499597457606136, 34.84570025054353, 36.07654166938045, 36.899884804719484]
  P: Array{Float64}((256, 256)) [0.1449279306214216 0.005919536496859418 … 0.00196042283590344 0.005370199356810865; 0.0977684655406359 -0.04973701370055792 … -0.0878992312227967 -0.05778779773302181; … ; 0.04308743597446929 0.04324139883354045 … -0.019127423566981328 -0.04138098814577151; -0.13325192873325928 -0.028583547120122572 … 0.12277657998369695 -0.056353074728779456]
  invP: Adjoint{Float64, Matrix{Float64}}
    parent: Array{Float6

In [35]:
M2 ≈ parent(edM2) == Matrix(edM2)

true

In [36]:
M2 ≈ edM2

true

In [37]:
c2*M2 ≈ c2*edM2 ≈ edM2*c2

true

In [38]:
c2\M2 ≈ c2\edM2 ≈ edM2/c2

true

In [39]:
(
    exp(M2)
    ≈ exp(edM2)
    ≈ EigenDecomposedMatrices.exp_old(edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2)
    ≈ EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2)
)

true

In [40]:
@show typeof(y2)
(
    exp(M2) * v2
    ≈ exp(edM2) * v2
    ≈ EigenDecomposedMatrices.exp_old(edM2) * v2
    ≈ mul!(y2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2), v2)
    ≈ mul!(y2, EigenDecomposedMatrices.exp_eigendecomposed!(Y2, edM2, expE2, tmpY2), v2)
)

typeof(y2) = Vector{Float64}


true

In [41]:
@btime edM2 = EigenDecomposedMatrices.EigenDecomposed(M2);

  6.472 ms (14 allocations: 1.59 MiB)


In [42]:
@btime exp($M2) * $v2
@btime exp($edM2) * $v2
@btime EigenDecomposedMatrices.exp_old($edM2) * $v2
@btime mul!($y2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2), $v2)
@btime mul!($y2, EigenDecomposedMatrices.exp_eigendecomposed!($Y2, $edM2, $expE2, $tmpY2), $v2);

  7.427 ms (19 allocations: 2.60 MiB)
  726.000 μs (6 allocations: 1.00 MiB)
  719.700 μs (8 allocations: 1.01 MiB)
  690.300 μs (3 allocations: 514.17 KiB)
  662.900 μs (0 allocations: 0 bytes)
